# RNN for Sentiment Analysis (IMDB movie Reviews)

### Step 0: Setup and Imports

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Step 1: Load and Preprocess Data